In [22]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
%autoreload 
import sys
sys.path.append('../')

import numpy as np
import pandas as pd

from pycaret.regression import *

# Import data

In [47]:
df = pd.read_csv('../data_feature_enigineering/df_selected_preprocessed_mean.csv')
df = df.drop(columns=['date_forecast'])

In [48]:
# show nan values
df.isnull().sum()

direct_rad:W                   0
sun_elevation:d                0
visibility:m                   0
pressure_100m:hPa              0
relative_humidity_1000hPa:p    0
day_of_year                    0
air_density_2m:kgm3            0
ceiling_height_agl:m           0
wind_speed_u_10m:ms            0
wind_speed_v_10m:ms            0
sun_azimuth:d                  0
day                            0
wind_speed_10m:ms              0
effective_cloud_cover:p        0
day_of_week                    0
fresh_snow_24h:cm              0
fresh_snow_3h:cm               0
building                       0
pv_measurement                 0
dtype: int64

# PyCaret setup

In [49]:
s = setup(df, target = 'pv_measurement', categorical_features=['building'], session_id=3, preprocess=False, )

,Description,Value
0,Session id,3
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(92951, 19)"
4,Transformed data shape,"(92951, 19)"
5,Transformed train set shape,"(65065, 19)"
6,Transformed test set shape,"(27886, 19)"
7,Numeric features,17
8,Categorical features,1


## Want to compare different tree models as they work well with this kinda problem

In [50]:
compared_models = compare_models(include = ['dt', 'rf', 'et', 'gbr', 'xgboost', 'lightgbm', 'catboost', 'mlp', 'ada'], n_select=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,90.0495,62411.9970,249.6996,0.8945,1.7398,2.4388,7.2320
xgboost,Extreme Gradient Boosting,93.5342,66268.1855,257.3089,0.8881,1.6781,4.5776,0.1850
et,Extra Trees Regressor,86.0562,66793.2800,258.2828,0.8872,0.9308,2.0094,3.0620
lightgbm,Light Gradient Boosting Machine,91.4516,67456.5182,259.5959,0.8860,1.4380,3.1145,195.5470
rf,Random Forest Regressor,87.6967,68385.1113,261.3260,0.8845,0.9845,2.1144,9.8340
gbr,Gradient Boosting Regressor,109.1628,81983.0105,286.1873,0.8615,1.9726,2.9284,3.0340
ada,AdaBoost Regressor,169.4108,127862.8648,357.2364,0.7841,2.6466,10.1219,1.0270
mlp,MLP Regressor,170.0252,133992.7331,364.2854,0.7738,2.8601,19.0269,6.5750
dt,Decision Tree Regressor,116.9701,137235.2674,370.2612,0.7679,1.0053,1.3874,0.2950


In [54]:
compared_models

 XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=3, ...),
 ExtraTreesRegressor(n_jobs=-1, random_state=3),
 LGBMRegressor(n_jobs=-1, random_state=3),
 RandomForestRegressor(n_jobs=-1, random_state=3)]

In [55]:
# save the most promising models
catboost = compared_models[0]
extra_trees = compared_models[2]

In [57]:
save_model(catboost, '/home/henrikraa/models/02H_pycaret_basic_catboost_mean_FE_1')
save_model(extra_trees, '/home/henrikraa/models/02H_pycaret_basic_extra_trees_mean_FE_1')

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('trained_model',
                  ExtraTreesRegressor(n_jobs=-1, random_state=3))]),
 '/home/henrikraa/models/02H_pycaret_basic_extra_trees_mean_FE_1.pkl')

In [59]:
tuned_extra_trees = tune_model(extra_trees, optimize='MAE', search_library = 'optuna', return_tuner=True)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:26:26
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Results
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


[I 2023-11-02 22:26:26,744] Searching the best hyperparameters using 65065 samples...
[I 2023-11-03 02:35:33,267] Finished hyperparameter search!
